# What is a Query Router?
A query router intelligently selects the best path for a query based on its content, balancing factors like complexity, cost, latency, and domain relevance 

In this node, I have done a small implementation using Langchain, LlamaIndex, DsPY.

# Routing Strategies
## LLM-based Selector
 - Use an LLM prompt to choose among paths (vector or keyword index, search tool, etc.) 

## Function-Calling
 - Use structured tools via function call APIs, letting the LLM pick which to invoke 

## Semantic Router
 - Embed query and compare to example embeddings for each route using similarity search 

## Classification Router
 - Train a classifier (LLM or lightweight model) to map queries to the right route 

In [1]:
%pip install langchain llama-index langchain-anthropic openai anthropic


INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.8/735.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.6.0
    Uninstalling setuptools-75.6.0:
      Successfully uninstalled setuptools-75.6.0
  Attempting uninstall: pypdf
    Found existing installation: pypdf 4.2.0
    Uninstalling pypdf-4.2.0:
      Successfully uninstalled pypdf-4.2.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninsta

In [16]:
%pip install llama-index



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
OPENAI_API_KEY="sk-proj-1234567890"
ANTHROPIC_API_KEY="sk-ant-api03-1234567890"


## Base Tools

In [ ]:
def web_search_tool(query):
    return f"Web results for: {query}"

def sql_tool(query):
    return f"SQL executed: {query}"

def calculator_tool(query):
    return f"Calculation result: {eval(query)}"

def weather_tool(location):
    return f"Weather at {location}: 25°C, Sunny"


## Langchain based query router

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

# Prompts per tool
prompt_infos = [
    {
        "name": "calculator",
        "description": "Handles math and calculations",
        "prompt_template": PromptTemplate.from_template("You are a calculator. Solve: {input}")
    },
    {
        "name": "web_search",
        "description": "Handles general information search",
        "prompt_template": PromptTemplate.from_template("Search online for: {input}")
    },
    {
        "name": "sql",
        "description": "Handles SQL-like data queries",
        "prompt_template": PromptTemplate.from_template("Query a database: {input}")
    },
    {
        "name": "weather",
        "description": "Handles weather queries",
        "prompt_template": PromptTemplate.from_template("Provide weather info for: {input}")
    },
]

from langchain.chains.router import MultiPromptChain
router_chain = MultiPromptChain.from_prompts(llm=llm, prompt_infos=prompt_infos) ## LLM based

response = router_chain.run("What's the weather in Tokyo?")
print(response)


## LlamaIndex QueryEngineRouter

In [ ]:
from llama_index.tools import QueryEngineTool
from llama_index.query_engine.router import QueryEngineRouter
from llama_index.llms import OpenAI
from llama_index.query_engine import CustomQueryEngine

llm = OpenAI(model="gpt-3.5-turbo")

class CustomTool(CustomQueryEngine):
    def __init__(self, func):
        self.func = func
    def query(self, input):
        return self.func(input)

tools = [
    QueryEngineTool.from_defaults(
        query_engine=CustomTool(web_search_tool),
        metadata={"name": "Web Search", "description": "Searches the web"}
    ),
    QueryEngineTool.from_defaults(
        query_engine=CustomTool(weather_tool),
        metadata={"name": "Weather", "description": "Gives weather info"}
    ),
    QueryEngineTool.from_defaults(
        query_engine=CustomTool(calculator_tool),
        metadata={"name": "Calculator", "description": "Does calculations"}
    ),
    QueryEngineTool.from_defaults(
        query_engine=CustomTool(sql_tool),
        metadata={"name": "SQL", "description": "Query structured data"}
    )
]

router = QueryEngineRouter(query_engine_tools=tools, selector="llm") 
response = router.query("What's 5 * 12 + 100?")
print(response)


## DSPy Router Example

In [ ]:
import dsp

options = ["calculator", "weather", "web_search", "sql"]

router = dsp.Select(options=options)

query = "How many kilometers are in 100 miles?"
tool = router(query)
print(f"Selected Tool: {tool}")

# Call corresponding function
tool_map = {
    "calculator": calculator_tool,
    "weather": weather_tool,
    "web_search": web_search_tool,
    "sql": sql_tool,
}
response = tool_map[tool](query) 
print(f"Response: {response}")
